


**2024.11.20**

#데이터 전처리 2 및 모델 학습




    1. 관광지명
    2. 소재지(도로명 주소)
    3. 공공편익시설정보, 휴양 및 문화시설정보
    4. 관광지소개



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

###1. 말뭉치 데이터 전처리

In [ ]:
import json
import pandas as pd

# Google Drive에 저장한 json 파일 불러오기
import json

from google.colab import drive
drive.mount('/content/drive')
file_path = '/content/drive/MyDrive/Colab Notebooks/전국관광지정보표준데이터.json'

with open(file_path, 'r') as file:
    data = json.load(file)

# 필요한 정보만 추출
processed_data = [
    {
        "name": item["관광지명"],
        "region": item.get("소재지(도로명 주소)"),
        "category": " / ".join(filter(None, [item.get("공공편익시설정보"), item.get("휴양 및 문화시설정보")])) or "정보 없음"
        "description": item.get("관광지소개")
    }
    for item in data["items"]
]

# 데이터 프레임으로 변환 (시각적 확인 및 처리 용이)
df = pd.DataFrame(processed_data)

# 전처리 완료된 데이터 저장
file_path = "/content/sample_data/processed_travel_data.json"

with open(file_path, "w", encoding='utf-8') as file:
    json.dump(df, file)
    print("데이터 전처리 JSON 파일이 성공적으로 저장되었습니다.")



###2. 모델 학습 데이터 준비

In [ ]:
# Prompt와 Response 생성
prompt_response_pairs = [
    {
        "prompt": f"{row['region']}에서 갈만한 {row['category']} 장소 추천해줘.",
        "response": f"{row['name']} - {row['description']}"
    }
    for _, row in df.iterrows()
]

# 학습 데이터 저장
file_path = "/content/sample_data/travel_chatbot_data.json"

with open(file_path, "w", encoding='utf-8') as file:
    json.dump(prompt_response_pairs, file)
    print("학습 데이터 JSON 파일이 성공적으로 저장되었습니다.")

# 샘플 데이터 확인
print(prompt_response_pairs[:2])


###3. 모델 학습 및 추론

In [ ]:
from transformers import GPT2LMHeadModel, PreTrainedTokenizerFast

# KoGPT 모델 및 토크나이저 로드
model_name = "skt/kogpt2-base-v2"
tokenizer = PreTrainedTokenizerFast.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

# Prompt 예제
prompt = "서울에서 갈만한 자연 장소 추천해줘."

# 입력 토큰 생성
input_ids = tokenizer.encode(prompt, return_tensors='pt')

# KoGPT로 텍스트 생성
output = model.generate(
    input_ids,
    max_length=50,   # 생성할 텍스트 길이
    num_return_sequences=1,
    do_sample=True,  # 랜덤 샘플링 활성화
    top_k=50,        # 상위 K개의 토큰만 고려
    top_p=0.95       # 상위 P 확률의 토큰만 고려
)

# 결과 디코딩
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
print("Generated Text:", generated_text)


###4. Fine-Tuning

In [ ]:
from datasets import Dataset

# 데이터셋 로드
with open('travel_chatbot_data.json', 'r') as file:
    data = json.load(file)

# Hugging Face 데이터셋 변환
dataset = Dataset.from_dict({
    "prompt": [item["prompt"] for item in data],
    "response": [item["response"] for item in data]
})


In [ ]:
from transformers import Trainer, TrainingArguments

# 학습 데이터셋 준비
def preprocess_function(examples):
    inputs = [f"Q: {q}\nA:" for q in examples["prompt"]]
    targets = examples["response"]
    model_inputs = tokenizer(inputs, padding="max_length", truncation=True, max_length=50)
    labels = tokenizer(targets, padding="max_length", truncation=True, max_length=50)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

processed_dataset = dataset.map(preprocess_function, batched=True)

# TrainingArguments 설정
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=4,
    num_train_epochs=3,
    save_steps=500,
    save_total_limit=2,
    fp16=True
)

# Trainer 생성
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=processed_dataset,
    tokenizer=tokenizer
)

# Fine-Tuning 시작
trainer.train()


###5. 추론

In [ ]:
# Fine-Tuned 모델 로드
model.save_pretrained('./fine_tuned_kogpt')
tokenizer.save_pretrained('./fine_tuned_kogpt')

# Fine-Tuned 모델 로드 후 추론
fine_tuned_model = GPT2LMHeadModel.from_pretrained('./fine_tuned_kogpt')
fine_tuned_tokenizer = PreTrainedTokenizerFast.from_pretrained('./fine_tuned_kogpt')

# Fine-Tuned 모델 사용
prompt = "서울에서 갈만한 문화 장소 추천해줘."
input_ids = fine_tuned_tokenizer.encode(prompt, return_tensors='pt')
output = fine_tuned_model.generate(input_ids, max_length=50, do_sample=True, top_k=50, top_p=0.95)
print(fine_tuned_tokenizer.decode(output[0], skip_special_tokens=True))
